<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#실패" data-toc-modified-id="실패-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>실패</a></span></li><li><span><a href="#영화별-크롤링" data-toc-modified-id="영화별-크롤링-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>영화별 크롤링</a></span></li></ul></div>

# 실패
- 1000P까지만 가능
- 네이버에서 막아놓음
- 다른 방법으로 크롤링하자!

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
def max_page(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    return int(soup.find('strong', class_='c_88 fs_11').text) // 10 + 1

url = 'https://movie.naver.com/movie/point/af/list.nhn'
def crawling_data(url):
    df = pd.DataFrame(columns = ['title_num', 'title', 'review', 'auther'])
    max_p = max_page(url)
    error_li = []
    for i in range(1, max_p):
        if i % 10 == 0:
            print(i)
        params = {'page' : i}
        res = requests.get(url, params = params)
        soup = BeautifulSoup(res.content, 'html.parser')

        tr_li = soup.find('tbody').find_all('tr')
        for idx, tr in enumerate(tr_li):
            try:
                now = dict()
                now['title_num'] = tr.find('td', class_='ac num').get_text()
                now['title'] = tr.find('a', class_='movie color_b').get_text()
                now['review'] = ''.join(tr.find('a', class_='report').get('href').split(",")[2:-2]).strip()[1:-1]
                now['auther'] = tr.find('a', class_='author').get_text()
                df = df.append(now, ignore_index = True)
            except:
                print('error', f"{i},{idx}")
                error_li.append([i, idx])

    return df, error_li


# 영화별 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
def max_page(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    q, r = divmod(int(soup.find('strong', class_='c_88 fs_11').text), 10)
    return q+1 if r else q

def crawling_data_by_movie(number):
    number = int(number)
    base_url = 'https://movie.naver.com/movie/point/af/list.nhn?st=mcode&' + f"sword={number}" + '&target=after'
    max_p = max_page(base_url) + 1
    
    df = pd.DataFrame(columns = ['number', 'title', 'score', 'review', 'auther'])
    error_li = []
    for i in range(1, max_p):
#         if i % 100 == 0:
#             print(i)
        params = {'page' : i}
        res = requests.get(base_url, params = params)
        soup = BeautifulSoup(res.content, 'html.parser')

        tr_li = soup.find('tbody').find_all('tr')
        for idx, tr in enumerate(tr_li):
            try:
                now = dict()
                now['number'] = tr.find('td', class_='ac num').get_text()
                now['title'] = tr.find('a', class_='movie color_b').get_text()
                now['score'] = tr.find('em').get_text()
                now['review'] = ''.join(tr.find('a', class_='report').get('href').split(",")[2:-2]).strip()[1:-1]
                now['auther'] = tr.find('a', class_='author').get_text()
                df = df.append(now, ignore_index = True)
            except:
                print('error', f"{i},{idx}")
                error_li.append([i, idx])

    return df, error_li

In [ ]:
a, b = crawling_data_by_movie(11206)
print(b)
print(a.head())

In [ ]:
len(a)

In [ ]:
# 10001이 처음이고 199900이 마지막임 11월 6일 기준
df = pd.DataFrame(columns = ['number', 'title', 'score', 'review', 'auther'])
for num in range(10_001, 199_901):
    now, error = crawling_data_by_movie(num)
    if len(now) > 100:
        print(num, error)
        df.append(now, ignore_index = True)
df.head()